In [ ]:
import ee

In [ ]:
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project="sona-ee")

In [ ]:
map = geemap.Map(ee_Initialise=False)
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
mumbai = ee.FeatureCollection('projects/sona-ee/assets/boundary')
map.addLayer(mumbai,{},'boundary')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
roads = ee.FeatureCollection('projects/sona-ee/assets/roads')
map.addLayer(roads,{},'roads')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
railways = ee.FeatureCollection('projects/sona-ee/assets/railways')
map.addLayer(railways,{},'railways')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
waterways = ee.FeatureCollection('projects/sona-ee/assets/waterways')
map.addLayer(waterways,{},'waterways')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
landuse = ee.FeatureCollection('projects/sona-ee/assets/Landuse')
map.addLayer(landuse,{},'landuse')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
waterbodies = ee.FeatureCollection('projects/sona-ee/assets/waterbodies')
map.addLayer(waterbodies,{},'waterbodies')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
buildings = ee.FeatureCollection('projects/sona-ee/assets/buildings')
map.addLayer(buildings,{},'buildings')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
import geopandas as gpd

In [ ]:
gjs = gpd.read_file("/content/BOUNDARY.shp")

DataSourceError: /content/BOUNDARY.shp: No such file or directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gjs.to_file("boundary.geojson",driver="GeoJSON")

In [ ]:
gjs = gpd.read_file("/content/Landuse.shp")

In [ ]:
gjs.to_file("Landuse.geojson",driver="GeoJSON")

In [ ]:
gjs = gpd.read_file("/content/railways.shp")

DataSourceError: /content/railways.shp: No such file or directory

In [ ]:
gjs.to_file("railways.geojson",driver="GeoJSON")

In [ ]:
gjs = gpd.read_file("/content/roads.shp")

In [ ]:
gjs.to_file("roads.geojson",driver="GeoJSON")

In [ ]:
gjs = gpd.read_file("/content/waterbodies.shp")

In [ ]:
gjs.to_file("waterbodies.geojson",driver="GeoJSON")

In [ ]:
gjs = gpd.read_file("/content/waterways.shp")

In [ ]:
gjs.to_file("waterways.geojson",driver="GeoJSON")

In [ ]:
gjs = gpd.read_file("/content/Landuse.shp")

In [ ]:
gjs.to_file("landuse.geojson",driver="GeoJSON")

In [ ]:
DEM = ee.Image("CGIAR/SRTM90_V4")
Mumbai_DEM = DEM.clip(mumbai)
map.addLayer(Mumbai_DEM,{},'DEM')
map

Map(bottom=117096.0, center=[19.233363381183896, 73.1902313232422], controls=(WidgetControl(options=['position…

In [ ]:
NDVI = ee.ImageCollection("LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI")
Mumbai_NDVI = NDVI.filterDate("2025-01-01","2025-12-01").mosaic().clip(mumbai)
map.addLayer(Mumbai_NDVI,{'palette':["#8B0000","#FF4500","#FFD700","#ADFF2F","#006400"]},'NDVI')
map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=Mumbai_NDVI,
    description='Mumbai_NDVI_Export',
    folder='GEE_Exports',
    fileNamePrefix='Mumbai_NDVI',
    scale=30,
    region=mumbai.geometry().bounds(),
    fileFormat='GeoTIFF'
)
task.start()
print('Export task started for Mumbai_NDVI. Check your Google Drive for the GeoTIFF.')

Export task started for Mumbai_NDVI. Check your Google Drive for the GeoTIFF.


In [ ]:
task = ee.batch.Export.image.toDrive(
    image=Mumbai_DEM,
    description='Mumbai_DEM_Export',
    folder='GEE_Exports',
    fileNamePrefix='Mumbai_DEM',
    scale=90,
    region=mumbai.geometry().bounds(),
    fileFormat='GeoTIFF'
)
task.start()
print('Export task started for Mumbai_DEM. Check your Google Drive for the GeoTIFF.')

Export task started for Mumbai_DEM. Check your Google Drive for the GeoTIFF.


In [ ]:
current_map_bounds = ee.Geometry.Rectangle(map.bounds)

task_zoomed_dem = ee.batch.Export.image.toDrive(
    image=Mumbai_DEM,
    description='Mumbai_DEM_Zoomed_Export',
    folder='GEE_Exports',
    fileNamePrefix='Mumbai_DEM_Zoomed',
    scale=10,
    region=current_map_bounds,
    fileFormat='GeoTIFF'
)
task_zoomed_dem.start()
print('Export task started for Mumbai_DEM_Zoomed. Check your Google Drive for the GeoTIFF.')
# The task.finish() method does not exist. The task runs asynchronously after task.start().
# If you need to wait for completion, use task.wait_for_completion().
print('Export task completed for Mumbai_DEM_Zoomed. Check your Google Drive for the GeoTIFF.')

Export task started for Mumbai_DEM_Zoomed. Check your Google Drive for the GeoTIFF.
Export task completed for Mumbai_DEM_Zoomed. Check your Google Drive for the GeoTIFF.


In [ ]:
status_zoomed_dem = task_zoomed_dem.status()
print(f"Current status of Mumbai_DEM_Zoomed_Export: {status_zoomed_dem['state']}")

if status_zoomed_dem['state'] == 'COMPLETED':
    print('The export task for Mumbai_DEM_Zoomed has completed successfully.')
    if 'destination_uris' in status_zoomed_dem and status_zoomed_dem['destination_uris']:
        print(f"You should find the file(s) in this Google Drive folder: {status_zoomed_dem['destination_uris'][0]}")
    else:
        print('No destination URI found in the task status. Please check your Google Drive manually under the folder "GEE_Exports".')
elif status_zoomed_dem['state'] == 'RUNNING':
    print('The export task is still running. Please wait a bit longer and check again.')
elif status_zoomed_dem['state'] == 'READY':
    print('The export task is queued and waiting to start.')
elif status_zoomed_dem['state'] == 'FAILED':
    error_message = status_zoomed_dem.get('error_message', 'No specific error message provided.')
    print(f"The export task failed with the following error: {error_message}")
    print('Please review the error message and ensure your Google Drive has enough space and correct permissions.')
else:
    print('Unknown task status. Please try refreshing or checking the Earth Engine Tasks tab.')

Current status of Mumbai_DEM_Zoomed_Export: FAILED
The export task failed with the following error: Export too large: specified 226052578908 pixels (max: 100000000). Specify higher maxPixels value if you intend to export a large area.
Please review the error message and ensure your Google Drive has enough space and correct permissions.
